<a href="https://colab.research.google.com/github/maelleal/TCC_Recomendacao_INE/blob/main/Model_based_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-learn
!pip install -U scikit-surprise
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pandas
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD

In [9]:
dataset = pandas.read_excel('/content/drive/My Drive/TCC - Sistema recomendação INE/tabela_areas_como_itens.xlsx')
dataset = dataset.replace(["#N/A"], 0)
dataset_filtrado = dataset.replace(np.nan, 0)
dataset_filtrado = dataset_filtrado.round(2)
dataset_filtrado.head()
print(dataset_filtrado.dtypes)
print(dataset_filtrado)

MATRICULA     float64
COD_AREA      float64
MEDIA_NOTA    float64
dtype: object
       MATRICULA  COD_AREA  MEDIA_NOTA
0     11100878.0       1.0        9.75
1     12200660.0       1.0        8.25
2     12101248.0       1.0        8.30
3      9232055.0       1.0        9.75
4     12101265.0       1.0        7.30
...          ...       ...         ...
2502  16100725.0      13.0        8.50
2503  16100728.0      13.0        9.13
2504  16101110.0      13.0        0.00
2505  16105151.0      13.0        9.40
2506  16105545.0      13.0        0.00

[2507 rows x 3 columns]


In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

'''
the user_item_matrix will look like this
|        | item 1 | ... | item m |
|--------|--------|-----|--------|
| user 1 | 3      | 0   | 0      |
| ...    | 0      | 4   | 5      |
| user n | 2      | 0   | 0      |
'''

# initial hyperparameter
epsilon = 1e-9
n_latent_factors = 10

# generate item lantent features
item_svd = TruncatedSVD(n_components = n_latent_factors)
item_features = item_svd.fit_transform(dataset_filtrado.transpose()) + epsilon


# generate user latent features
user_svd = TruncatedSVD(n_components = n_latent_factors)
user_features = user_svd.fit_transform(dataset_filtrado) + epsilon

In [26]:
from surprise import SVD, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.split import train_test_split

'''
funk MF

rating dataframe will look like this
| user_id | item_id | rating          |
|---------|---------|-----------------|
| 1       | 1       | 5               |
| ...     | ...     | ...             |
| n       | m       | 3               |
'''

# preprocessing the data
reader = Reader(rating_scale=(1,10))


# dataset com 3 colunas
data = Dataset.load_from_df(dataset_filtrado[['MATRICULA','COD_AREA','MEDIA_NOTA']], reader)

train, test = train_test_split(data, test_size=.2, random_state=42)

# initial model
algo = SVD(random_state = 42)
algo.fit(train)
pred = algo.test(test)

# RMSE = Raiz quadrada do erro-médio
# MAE = erro absoluto médio

# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# evaluate the rmse result of the prediction and ground thuth
accuracy.rmse(pred)

uid = str(16105151.0)  # raw user id (as in the ratings file). They are **strings**!
iid = str(13)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

RMSE: 1.7643
user: 16105151.0 item: 13         r_ui = 4.00   est = 5.47   {'was_impossible': False}


In [16]:
from surprise import SVD, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.split import train_test_split
from surprise import KNNBasic

# preprocessing the data
reader = Reader(rating_scale=(1,10))


# dataset com 3 colunas
data = Dataset.load_from_df(dataset_filtrado[['MATRICULA','COD_AREA','MEDIA_NOTA']], reader)
trainset = data.build_full_trainset()

algo = KNNBasic()
algo.fit(trainset)

uid = str(12200660)  # raw user id (as in the ratings file). They are **strings**!
iid = str(1)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
user: 12200660   item: 1          r_ui = 4.00   est = 5.47   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
